In [ ]:
'''Read train/validate data from separated CSV files.'''
# Read data using Pandas

import pandas as pd

train_set = pd.read_csv("../1. 초기데이터/Data_Set_LDL-C_trainset.csv")
val_set = pd.read_csv("../1. 초기데이터/Data_Set_LDL-C_valset.csv")

In [ ]:
'''Divide train data and test data. (X_data is train data Y_data is test data.)'''
import numpy as np

# Extract Train Data
# x_train is data for train DNN.
x_train = train_set.iloc[:,[3,4,6]]
x_train = np.array(x_train)

# y_train is data for evaluate output of DNN.
y_train = train_set.iloc[:,[5]]
y_train = np.array(y_train)

# Extract Test Data
# x_test is data for train DNN.
x_val = val_set.iloc[:,[3,4,6]]
x_val = np.array(x_val)

# y_test is data for evaluate output of DNN.
y_val = val_set.iloc[:,[5]]
y_val = np.array(y_val)

In [ ]:
'''Data Normalization'''

mean = x_train.mean(axis=0)
std = x_train.std(axis=0)

x_train -= mean
x_train /= std

x_val -= mean
x_val /= std

In [ ]:
'''Configuration of model'''
from tensorflow.keras import models,layers
from keras import backend as K

# User defined loss function
def squared_error(y_true, y_pred):
    return (float(y_true) - float(y_pred)) ** 2

# User defined metrics
# Reference: https://stackify.dev/168133-kerasregressor-coefficient-of-determination-r2-score
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Build DNN model.
def build_model(): 
    model = models.Sequential()
    model.add(layers.Dense(30, activation='relu',
                           input_dim=3))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dense(1, activation='relu'))
    model.compile(optimizer='adam', loss=squared_error, metrics=[coeff_determination])
    return model

In [ ]:
'''K-fold cross validation'''
# Reference: https://morningcoding.tistory.com/entry/Keras%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EB%B3%B4%EC%8A%A4%ED%84%B4-%EC%A3%BC%ED%83%9D%EA%B0%80%EA%B2%A9-%EC%98%88%EC%B8%A1-K-Fold

import numpy as np
import time

# Hyperparameter of this train
k = 5
num_epochs = 100
batch = 80

num_val_samples = len(x_train) // k

all_r2_histories = []
all_scores = []
start_time = time.time() 


for i in range(k):
    print('Processing Fold #', i)
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]  
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(  
        [x_train[:i * num_val_samples],
         x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)

    model = build_model()  
    history = model.fit(partial_train_data, partial_train_targets, 
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=batch, verbose=1)
    r2_history = history.history['coeff_determination']
    all_r2_histories.append(r2_history)
end_time = time.time()
print("Running Time: ",end_time-start_time)

In [ ]:
average_r2_history = [
    np.mean([x[i] for x in all_r2_histories]) for i in range(num_epochs)]

In [ ]:
"""Visualization of train result"""
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_r2_history) + 1), average_r2_history)
plt.xlabel('Epochs')
plt.ylabel('R2')
plt.show()

In [ ]:
"""Visualization convert to smooth curve"""
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

smooth_r2_history = smooth_curve(average_r2_history[10:])

plt.plot(range(1, len(smooth_r2_history) + 1), smooth_r2_history)
plt.xlabel('Epochs')
plt.ylabel('R2')
plt.show()

In [ ]:
'''Train model using validate set data'''
num_epochs = 100
batch = 80

model = build_model()  
model.fit(x_val, y_val,  
          epochs=num_epochs, batch_size=batch, verbose=1)
val_mse_score, val_mae_score = model.evaluate(x_val, y_val)
print(val_mse_score)
print(val_mae_score)

In [ ]:
'''Assume predict value'''
x_hat = x_val
y_hat = model.predict(x_hat, use_multiprocessing=True)
print('Estimate : ',y_hat)
print(x_hat.shape)
print(y_hat.shape)

In [ ]:
from sklearn.metrics import r2_score
import math

# Calculate Determination Coefficient
r2 = r2_score(y_val, y_hat)
print('R2 : ', r2)
# Calculate Correlation Coefficient
r = math.sqrt(r2)
print('R : ', r)

In [ ]:
'''Save predict_data(y_hat) to Data_Set_LDL-C-DNN.csv'''
df_CD = pd.DataFrame(y_hat,columns=['DNN'])
df_CD.to_csv('../1. 초기데이터/Data_Set_LDL-C-DNN.csv')